In [27]:
import os
import sys
import json
from dotenv import load_dotenv
import google.generativeai as palm
from pprint import pprint

sys.path.append('..')
from backend.DB.db import Database

In [15]:
load_dotenv()
api_key = os.getenv('PALM_API_KEY')
palm.configure(api_key=api_key)

In [22]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


In [47]:
db = Database()

sql = '''
SELECT topic_name FROM topics
WHERE category = "hotel";
'''

topics = db.query_sql(sql)
topics = [item[0] for item in topics]

with open(os.path.join('backend', 'gpt-instructions.txt'), 'r') as FILE:
    instructions = FILE.read()

instructions = instructions.replace('%%topic_list%%', str(topics))
print(instructions)

Your task is to analyze and summarize salient points under each topic from
a review. These are your tasks:

* Indicate if a topic is mentioned in the review: true or false

* Evaluate if the guest was angry about that aspect of their stay:  true or false.

* Evaluate the sentiment on a scale of 1 to 5. Scale is:

0 - Not applicable
1 - very negative
2 - negative
3 - neutral
4 - positive
5 - very positive

* Summarize the guest's comments on the topic, in 1-2 sentences.

Analyze the review below as instructed above, delimited by triple 
ticks. Format as JSON, exactly as follows. Include all key/value pairs.

{topic : {"Relevant": value, "Angry": value, "Sentiment": value, "Summary": summary}}

Topics = ['amenities', 'frontdesk service', 'hotel condition', 'housekeeping', 'value']



In [57]:
results = db.query('raw_reviews')
review = f"{results[0]['title']}. {results[0]['review_text']}"

prompt = f'{instructions}\n\n======================\n\n ```{review}```'
completion = palm.generate_text(
    model=model,
    prompt=prompt,
    temperature=0,
    # The maximum length of the response
    max_output_tokens=800,
)

read_review = completion.result.replace('\'', '\"')
read_review = read_review.replace('\n', '')

json.loads(read_review)

{'amenities': {'Relevant': True,
  'Angry': False,
  'Sentiment': 4,
  'Summary': 'Room has kettle tv and fridge. Black out curtains were a good amenity.'},
 'frontdesk service': {'Relevant': False,
  'Angry': False,
  'Sentiment': 0,
  'Summary': ''},
 'hotel condition': {'Relevant': True,
  'Angry': False,
  'Sentiment': 4,
  'Summary': 'Overall room was clean and everything worked.'},
 'housekeeping': {'Relevant': True,
  'Angry': False,
  'Sentiment': 3,
  'Summary': 'Housekeeping only operates on alternate days.'},
 'value': {'Relevant': True,
  'Angry': False,
  'Sentiment': 3,
  'Summary': 'satisfactory experience but it’s priced on the higher side. Almost identical experiences can be had at lower prices elsewhere in moab.'}}